In [5]:
import pandas as pd

# Load data
df = pd.read_csv('../data/financial_data.csv')

# Quick analysis
print("Data Summary:")
print(df.groupby(['Type', 'Department'])['Amount'].sum().unstack())

# Create pivot table
report = df.pivot_table(
    index='Department',
    columns='Type',
    values='Amount',
    aggfunc='sum',
    margins=True
)
print("\nFinancial Report by Department:")
print(report)

Data Summary:
Department     Finance          IT   Marketing       Sales     Support
Type                                                                  
Expense     2027746.98  2863741.24  2890588.88  2435799.36  2703723.86
Revenue     2133145.89  2232794.62  2332316.26  2784697.77  2839928.76

Financial Report by Department:
Type            Expense      Revenue          All
Department                                       
Finance      2027746.98   2133145.89   4160892.87
IT           2863741.24   2232794.62   5096535.86
Marketing    2890588.88   2332316.26   5222905.14
Sales        2435799.36   2784697.77   5220497.13
Support      2703723.86   2839928.76   5543652.62
All         12921600.32  12322883.30  25244483.62


In [6]:
from statsmodels.tsa.arima.model import ARIMA

# Prepare monthly data
monthly_data = df.groupby(pd.to_datetime(
    df['Date']).dt.to_period('M'))['Amount'].sum()

# ARIMA Model
model = ARIMA(monthly_data, order=(1, 1, 1))
results = model.fit()

# 3-month forecast
forecast = results.forecast(steps=3)
print("3-Month Forecast:")
print(forecast)

3-Month Forecast:
2025-08    1.007712e+06
2025-09    9.733533e+05
2025-10    9.588584e+05
Freq: M, Name: predicted_mean, dtype: float64
